<a href="https://colab.research.google.com/github/Tarun-619/AI-lab-1WA23CS015/blob/main/AI_lab9_015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forward Chaining

In [1]:
def is_variable(x):
    return x[0].islower()

def unify(x, y, subst):
    if subst is None:
        return None
    if x == y:
        return subst
    if is_variable(x):
        return unify_var(x, y, subst)
    if is_variable(y):
        return unify_var(y, x, subst)
    return None

def unify_var(var, val, subst):
    if var in subst:
        return unify(subst[var], val, subst)
    new = subst.copy()
    new[var] = val
    return new


def parse_predicate(pred):
    name = pred[:pred.index("(")]
    args = pred[pred.index("(")+1 : pred.index(")")].split(", ")
    return name, args


def match_premises(premises, facts):
    subst = {}

    for prem in premises:
        prem_name, prem_args = parse_predicate(prem)
        found = False

        for fact in facts:
            fact_name, fact_args = parse_predicate(fact)

            if prem_name == fact_name and len(prem_args) == len(fact_args):
                temp = subst.copy()

                ok = True
                for p, f in zip(prem_args, fact_args):
                    result = unify(p, f, temp)
                    if result is None:
                        ok = False
                        break
                    temp = result

                if ok:
                    subst = temp
                    found = True
                    break

        if not found:
            return None

    return subst


def apply_substitution(pred, subst):
    name, args = parse_predicate(pred)
    new_args = [subst.get(a, a) for a in args]
    return f"{name}({', '.join(new_args)})"


def forward_chaining(facts, premises, conclusion, query):
    facts = set(facts)

    while True:
        subst = match_premises(premises, facts)
        if subst is None:
            break

        inferred = apply_substitution(conclusion, subst)

        if inferred not in facts:
            print(f"Inferred: {inferred}")
            facts.add(inferred)

            if inferred == query:
                return True, facts
        else:
            break

    return query in facts, facts

facts = {
    "Parent(Tanmay, Niddhi)",
    "Parent(Niddhi, Tejas)",
    "Male(Tanmay)",
    "Female(Niddhi)"
}

premises = [
    "Parent(x, y)",
    "Parent(y, z)"
]
conclusion = "Grandparent(x, z)"
query = "Grandparent(Tanmay, Tejas)"
result, derived = forward_chaining(facts, premises, conclusion, query)

print("\nFinal Derived Facts:")
for f in derived:
    print(f)

print("\nQuery:", query)
print("Result:", "TRUE  (Tanmay is Tejas's grandparent)" if result else "FALSE (Not provable)")


Inferred: Grandparent(Tanmay, Tejas)

Final Derived Facts:
Female(Niddhi)
Grandparent(Tanmay, Tejas)
Male(Tanmay)
Parent(Tanmay, Niddhi)
Parent(Niddhi, Tejas)

Query: Grandparent(Tanmay, Tejas)
Result: TRUE  (Tanmay is Tejas's grandparent)
